# Imports

In [40]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep

# Data

In [2]:
df = pd.read_excel('Small Town Murder Episodes.xlsx')

In [11]:
df.describe(include='all')

,Episode Number,Title,Duration (Min),Release Date,City,State/Province,Country,Zip Code,Population
count,107.000000,107,107.000000,107,106,105,106,4.000000,4.000000
unique,NaN,107,NaN,107,102,54,4,NaN,NaN
top,NaN,#032 - Broken Bones & Brutal Murder in Baraboo...,NaN,2017-12-13 00:00:00,Burlingame,AZ,USA,NaN,NaN
freq,NaN,1,NaN,1,2,3,102,NaN,NaN
first,NaN,NaN,NaN,2017-01-12 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2019-02-14 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,53.000000,NaN,116.065421,NaN,NaN,NaN,NaN,23513.250000,8159.500000
std,31.032241,NaN,39.550606,NaN,NaN,NaN,NaN,24387.321957,6033.010608
min,0.000000,NaN,2.000000,NaN,NaN,NaN,NaN,2767.000000,2775.000000
25%,26.500000,NaN,72.000000,NaN,NaN,NaN,NaN,2767.000000,3016.500000


In [7]:
df1 = df.dropna(subset=[
    'City',
    'State/Province',
    'Country'
], how='all')

In [10]:
df1.describe(include='all')

,Episode Number,Title,Duration (Min),Release Date,City,State/Province,Country,Zip Code,Population
count,106.000000,106,106.000000,106,106,105,106,4.000000,4.000000
unique,NaN,106,NaN,106,102,54,4,NaN,NaN
top,NaN,#032 - Broken Bones & Brutal Murder in Baraboo...,NaN,2017-12-13 00:00:00,Burlingame,AZ,USA,NaN,NaN
freq,NaN,1,NaN,1,2,3,102,NaN,NaN
first,NaN,NaN,NaN,2017-01-18 00:00:00,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,2019-02-14 00:00:00,NaN,NaN,NaN,NaN,NaN
mean,53.500000,NaN,117.141509,NaN,NaN,NaN,NaN,23513.250000,8159.500000
std,30.743563,NaN,38.132211,NaN,NaN,NaN,NaN,24387.321957,6033.010608
min,1.000000,NaN,58.000000,NaN,NaN,NaN,NaN,2767.000000,2775.000000
25%,27.250000,NaN,73.000000,NaN,NaN,NaN,NaN,2767.000000,3016.500000


# Make URLs

In [13]:
usa = df1.loc[
    df1.Country == 'USA',
    [
        'City',
        'State/Province'
    ]
].drop_duplicates().copy()

In [14]:
usa.describe()

,City,State/Province
count,100,100
unique,98,51
top,Warrenton,AZ
freq,2,3


In [15]:
usa['State/Province'].unique()

array(['MS', 'MA', 'MI', 'OH', 'TX', 'AZ', 'NY', 'KS', 'FL', 'CA', 'ME',
       'MT', 'VA', 'MN', 'CO', 'IL', 'WA', 'AL', 'NM', 'NJ', 'OK', 'TN',
       'CT', 'ID', 'GA', 'AR', 'WI', 'PA', 'MD', 'IN', 'LA', 'SD', 'KY',
       'NH', 'IA', 'UT', 'VT', 'NE', 'DE', 'AK', 'NC', 'NV', 'ND', 'MO',
       'SC', 'OR', 'WV', 'WY', 'RI', 'HI', 'California'], dtype=object)

In [21]:
# replacing "California" with "CA"
usa['State/Province'].replace({
    'California' : 'CA'
}, inplace=True)

In [23]:
usa.describe()

,City,State/Province
count,100,100
unique,98,50
top,Warrenton,AZ
freq,2,3


In [24]:
# join city and state to make url
usa['url'] = usa.City + ', ' + usa['State/Province']

In [28]:
# add wiki url
usa['url'] = 'https://en.wikipedia.org/wiki/' + usa.url

In [33]:
# replace ' ' with '_'
usa['url'] = usa.url.str.replace(' ', '_')

In [64]:
location = {}
for u in usa.url.unique():
    r = requests.get(u)
    if r.ok == True:
        soup = BeautifulSoup(r.text)
        location[u] = soup
        sleep(2)
    else:
        pass

In [71]:
usa1 = usa.merge(pd.DataFrame.from_dict(location, orient='index').reset_index().rename(columns={
    'index' : 'url',
    0 : 'scraped'
}), how='left')

In [89]:
eaton = usa1.loc[usa1.url == 'https://en.wikipedia.org/wiki/Eaton,_OH', 'scraped'].values[0]

In [96]:
eaton.find(name='div', attrs={
    'class' : 'postal-code'
}).text

'45320'

In [103]:
def ps(x):
    try:
        t = x.find(name='div', attrs={
            'class' : 'postal-code'
        }).text
        return t
    except AttributeError:
        return 'Check wiki page'

In [105]:
usa1['postal_code'] = usa1.scraped.dropna().apply(ps)

In [107]:
usa2 = usa1.drop(columns='scraped')

In [111]:
df1

,Episode Number,Title,Duration (Min),Release Date,City,State/Province,Country,Zip Code,Population
1,1,#001 - A Quadruple Murder in Mississippi,62,2017-01-18,Sarah,MS,USA,38665.0,2775.0
2,2,"#002 - A Kidnapping & Murder in Raynham, Massa...",58,2017-01-25,Raynham,MA,USA,2767.0,13383.0
3,3,"#003 - A Kidnapping & Murder in Raynham, Massa...",59,2017-02-01,Raynham,MA,USA,2767.0,13383.0
4,4,"#004 - Three Charred Bodies in Manistique, Mic...",61,2017-02-08,Manistique,MI,USA,49854.0,3097.0
5,5,"#005 - A Murder For Greed in Eaton, Ohio",58,2017-02-15,Eaton,OH,USA,NaN,NaN
6,6,#006 - The Best & Brightest Murdered in Hawkin...,63,2017-02-22,Hawkins,TX,USA,NaN,NaN
7,7,"#007 - A Young, Budding Serial Killer in St. J...",60,2017-03-01,St. Johns,AZ,USA,NaN,NaN
8,8,"#008 - A Trail Of Ashes & Bodies in Otisco, Ne...",61,2017-03-08,Otisco,NY,USA,NaN,NaN
9,9,#009 - Jealously Turns Into Multiple Murders i...,65,2017-03-15,Burlingame,KS,USA,NaN,NaN
10,10,#010 - A British Tourist Murdered in Monticell...,64,2017-03-22,Monticello,FL,USA,NaN,NaN


In [112]:
df2 = df.merge(usa2, how='left')

In [113]:
df2.head()

,Episode Number,Title,Duration (Min),Release Date,City,State/Province,Country,Zip Code,Population,url,postal_code
0,0,#000 - Preview,2,2017-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,#001 - A Quadruple Murder in Mississippi,62,2017-01-18,Sarah,MS,USA,38665.0,2775.0,"https://en.wikipedia.org/wiki/Sarah,_MS",NaN
2,2,"#002 - A Kidnapping & Murder in Raynham, Massa...",58,2017-01-25,Raynham,MA,USA,2767.0,13383.0,"https://en.wikipedia.org/wiki/Raynham,_MA",02767
3,3,"#003 - A Kidnapping & Murder in Raynham, Massa...",59,2017-02-01,Raynham,MA,USA,2767.0,13383.0,"https://en.wikipedia.org/wiki/Raynham,_MA",02767
4,4,"#004 - Three Charred Bodies in Manistique, Mic...",61,2017-02-08,Manistique,MI,USA,49854.0,3097.0,"https://en.wikipedia.org/wiki/Manistique,_MI",49854


In [114]:
df2.tail()

,Episode Number,Title,Duration (Min),Release Date,City,State/Province,Country,Zip Code,Population,url,postal_code
102,102,#102 - A Troubled & Troubling Past in Red Spri...,147,2019-01-16,Red Springs,NC,USA,NaN,NaN,"https://en.wikipedia.org/wiki/Red_Springs,_NC",28377
103,103,#103 - It's Always The Quiet Ones in Bonaparte...,148,2019-01-23,Bonaparte,IA,USA,NaN,NaN,"https://en.wikipedia.org/wiki/Bonaparte,_IA",52620
104,104,"#104 - The Trees Tell You To Murder in Dallas,...",153,2018-01-30,Dallas,OR,USA,NaN,NaN,"https://en.wikipedia.org/wiki/Dallas,_OR",97338
105,105,#105 - Charles Manson Without The LSD in Bloom...,155,2019-02-06,Bloomingdale,IL,USA,NaN,NaN,"https://en.wikipedia.org/wiki/Bloomingdale,_IL",60108
106,106,"#106 - Not Regulation Behavior in Burlingame, ...",129,2019-02-14,Burlingame,California,USA,NaN,NaN,NaN,NaN


In [118]:
df2['postal_code'] = df2.postal_code.str.split('-', expand=True)[0]

In [122]:
df2.postal_code = df2.postal_code.str.strip()

In [128]:
df2.postal_code = df2.postal_code.str.split('[', expand=True)[0]

In [133]:
df2.to_csv('Small Town Murder Episodes.csv', index=None)